In [1]:
import boto3
from botocore.config import Config
import os
import time
# Simple timer decorator, meant to show how long each step takes.
def timeit(function, *args, **kwargs):
    def wrapper(*args, **kwargs):
        start=time.perf_counter()
        func=function(*args, **kwargs)
        print(f"Finished {function.__name__} in {time.perf_counter()-start:.02f}s\n{'='*20}")
        return func
    return wrapper


In [30]:
class CloudHandler():

    def __init__(self):
        
        # General
        self.automation_tag = {
            'Key': 'DIP_AUTOMATION_BOTO',
            'Value': 'True',
        }
        self.filter_running_automation = [
            {
                'Name': 'tag:DIP_AUTOMATION_BOTO',
                'Values': [
                    'True',
                ]
            },
            {
                'Name': 'instance-state-name',
                'Values': [
                    'running',
                ]
            },            
        ]
        self.ubuntu20amiNorth = "ami-09e67e426f25ce0d7"
        self.ubuntu20amiSouth = "ami-00399ec92321828f5"
        self.delete_all = False

        # North Virginia
        self.cfg1 = Config(region_name="us-east-1") # Define region (default is us-east-1)
        self.North_ec2_resource = boto3.resource('ec2', config=self.cfg1) # make ec2 client
        with open("mysql.sh", "r") as f:
            self.script_db = f.read()
        with open("mysql.sql", "r") as f:
            self.script_db = self.script_db.replace("SCRIPT_SQL",f.read())
        with open("mysql.conf.d", "r") as f:
            self.script_db = self.script_db.replace("MYSQL_CONF",f.read())            

        # Ohio
        self.cfg2 = Config(region_name="us-east-2") # Define region (default is us-east-1)
        self.South_ec2_resource = boto3.resource('ec2', config=self.cfg2) # make ec2 client
        self.default_vpc_south = list(self.South_ec2_resource.vpcs.filter(Filters=[{'Name': 'is-default','Values': ['true']}]))[0].id
        with open("django.sh", "r") as f:
            self.script_django = f.read()

        self.ec2Client = boto3.client('ec2', config=self.cfg2)
        self.asgClient = boto3.client('autoscaling', config=self.cfg2)
        self.elbClient = boto3.client('elbv2', config=self.cfg2)
        self.rgtApiClient = boto3.client('resourcegroupstaggingapi', config=self.cfg2)
    
    # Returns db IP address (port is always 3306)
    def get_db_ip(self) -> str:
        return self.get_running_instances(self.North_ec2_resource)[0].public_ip_address

    # Updates values in django.sh to match dynamically generated information (such as mysql IP)
    def update_django_script(self):
        self.script_django = self.script_django.replace("s/node1/IPDB/g", f"s/node1/{self.get_db_ip()}/g", 1)

    # Forcibly deletes all known infrastructure
    def force_delete_all(self):
        
        self.delete_db()
        self.delete_autoscaling_group()
        self.delete_django()
        self.delete_elastic_load_balancer()

    # Set flag for deletion
    def ask_delete_all(self):
        if self.delete_all: return
        print("Would you like to delete all existing infrastructure? (y/n)")
        a = input()
        if a.strip().lower() not in ["y", "yes", ""]:
            print("Aborting...")
            self.delete_all = False
            return
        self.delete_all = True

    # Returns list of running instances that match the automation tag (for the specified ec2 resource)
    def get_running_instances(self, resource):
        return list(resource.instances.filter(Filters=self.filter_running_automation))

    # Returns list of available subnet ids
    def get_available_subnets(self):
        return [i["SubnetId"] for i in self.ec2Client.describe_subnets()["Subnets"]]

    # Delete MySQL db
    @timeit
    def delete_db(self):
        print("Deleting db...")
        filter=[
            {
                'Name': 'tag:DIP_AUTOMATION_BOTO',
                'Values': [
                    'True',
                ]
            },
        ]
        # Instances
        print("Destroying MySQL instance...")
        current_machines = self.North_ec2_resource.instances.filter(Filters=filter)
        destroy = [ins.terminate() for ins in current_machines]
        wait = [ins.wait_until_terminated() for ins in current_machines]

        # Sec Groups
        print("Destroying MySQL security group...")
        current_groups = self.North_ec2_resource.security_groups.filter(Filters=filter)
        destroy = [gr.delete() for gr in current_groups]    
        wait = [gr.wait_until_terminated() for gr in current_groups] 
        
        return 0

    # Delete django base
    @timeit
    def delete_django(self):
        filter=[
            {
                'Name': 'tag:DIP_AUTOMATION_BOTO',
                'Values': [
                    'True',
                ]
            },
        ]
        # Instances
        print("Destroying django base instance...")
        current_machines = self.South_ec2_resource.instances.filter(Filters=filter)
        destroy = [ins.terminate() for ins in current_machines]
        wait = [ins.wait_until_terminated() for ins in current_machines]
        
        # AMIs
        print("Destroying django_orm AMIs...")
        current_amis = self.South_ec2_resource.images.filter(Filters=filter)
        destroy = [ami.deregister() for ami in current_amis]    
        
        # # VPCs
        # try:
        #     current_subnets = self.South_ec2_resource.subnets.filter(Filters=filter)
        #     destroy = [sn.delete() for sn in current_subnets]
        # except Exception as e:
        #     print(f"Did not delete subnets. {e}")

        # # Subnets
        # try:
        #     current_vpc = self.South_ec2_resource.vpcs.filter(Filters=filter)
        #     destroy = [vpc.delete() for vpc in current_vpc]  
        #     self.South_vpc.delete()
        # except Exception as e:
        #     print(f"Did not delete vpcs. {e}")
        
        # Sec Groups
        print("Destroying django security group...")
        current_groups = self.South_ec2_resource.security_groups.filter(Filters=filter)
        destroy = [gr.delete() for gr in current_groups]    
        wait = [gr.wait_until_terminated() for gr in current_groups]
        return 0

    # Delete autoscaling group (if exists)
    @timeit
    def delete_autoscaling_group(self):
        try:
            self.asgClient.delete_auto_scaling_group(AutoScalingGroupName="asg_django", ForceDelete=True)
        except Exception as e:
            print(f"Unable to delete autoscaling group asg_django. {e}")
        try:
            self.ec2Client.delete_launch_template(LaunchTemplateName='django_template')
        except Exception as e:
            print(f"Unable to delete launch template django_template. {e}")

    # Delete ELB (if exists)
    @timeit
    def delete_elastic_load_balancer(self):
        print("Deleting elastic load balancer...")
        filter_elb=[
            {
                'Key': 'DIP_AUTOMATION_BOTO',
                'Values': [
                    'True',
                ]
            },
            {
                'Key': 'Name',
                'Values': [
                    'django-elb'
                ]
            }
        ]
        filter_tg=[
            {
                'Key': 'DIP_AUTOMATION_BOTO',
                'Values': [
                    'True',
                ]
            },
            {
                'Key': 'Name',
                'Values': [
                    'django-elb-tg'
                ]
            }
        ]
            
        response = self.rgtApiClient.get_resources(TagFilters=filter_elb)
        for resource in response["ResourceTagMappingList"]:
            arn_elb = resource['ResourceARN']
        response = self.rgtApiClient.get_resources(TagFilters=filter_tg)
        for resource in response["ResourceTagMappingList"]:
            arn_tg = resource['ResourceARN']
        try:
            self.elbClient.delete_load_balancer(LoadBalancerArn=arn_elb)
        except:
            print("Failed to delete ELB, could be ok.")

        try:
            self.elbClient.delete_target_group(TargetGroupArn=arn_tg)
        except:
            print("Failed to delete target group, could be ok.")
        
    # Creates security group for MySQL
    @timeit
    def _create_sec_group_db(self):
        security_group = self.North_ec2_resource.create_security_group(
        Description='Allow inbound traffic',
        GroupName='db',
        TagSpecifications=[
                {
                    'ResourceType': 'security-group',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'db'
                        },
                        self.automation_tag,
                    ]
                },
            ],
        )

        security_group.authorize_ingress(
            CidrIp='0.0.0.0/0',
            FromPort=22,
            ToPort=22,
            IpProtocol='tcp',
        )

        security_group.authorize_ingress(
            CidrIp='0.0.0.0/0',
            FromPort=3306,
            ToPort=3306,
            IpProtocol='tcp',
        )

        # By default all egress is allowed
        # security_group.authorize_egress(
        #     IpPermissions=[
        #             {
        #                 'FromPort': 3306,
        #                 'ToPort': 3306,
        #                 'IpProtocol': 'tcp',
        #                 'IpRanges': [
        #                     {
        #                         'CidrIp': '0.0.0.0/0',
        #                         'Description': 'All'
        #                     },
        #                 ]
        #             }
        #         ]
        # )
        security_group.load() # Commits updates
        self.mysql_sec_group = security_group
        return security_group
    
    # Creates MySQL instance
    @timeit
    def _create_instance_db(self, sec_group):
        instances = self.North_ec2_resource.create_instances(
        BlockDeviceMappings=[{
                    'DeviceName': '/dev/xvda',
                    'Ebs': {
                        'DeleteOnTermination': True,
                        'VolumeSize': 240,
                        'VolumeType': 'gp2'
                    },
                },
        ],
        ImageId=str(self.ubuntu20amiNorth),
        InstanceType='t2.micro',
        MaxCount=1,
        MinCount=1,
        KeyName="DIP",
        Monitoring={
            'Enabled': False
        },
        SecurityGroupIds=[
            sec_group.group_id,
        ],
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    self.automation_tag,
                    {
                        'Key': 'Name',
                        'Value': 'db',
                    },
                ],
            },
        ],
        UserData=self.script_db)
        instances[0].wait_until_running()
        return instances

    # Creates security group for all future django instances
    @timeit
    def _create_sec_group_django(self):
        security_group = self.South_ec2_resource.create_security_group(
        Description='Allow inbound traffic',
        GroupName='django',
        # VpcId=self.South_vpc.id,
        TagSpecifications=[
                {
                    'ResourceType': 'security-group',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'django'
                        },
                        self.automation_tag,
                    ]
                },
            ],
        )

        security_group.authorize_ingress(
            CidrIp='0.0.0.0/0',
            FromPort=22,
            ToPort=22,
            IpProtocol='tcp',
        )

        security_group.authorize_ingress(
            CidrIp='0.0.0.0/0',
            FromPort=8080,
            ToPort=8080,
            IpProtocol='tcp',
        )

        # By default all egress is allowed
        # security_group.authorize_egress(
        #     IpPermissions=[
        #             {
        #                 'FromPort': 8080,
        #                 'ToPort': 8080,
        #                 'IpProtocol': 'tcp',
        #                 'IpRanges': [
        #                     {
        #                         'CidrIp': '0.0.0.0/0',
        #                         'Description': 'All'
        #                     },
        #                 ]
        #             }
        #         ]
        # )
        security_group.load() # Commits updates
        self.django_sec_group = security_group
        return security_group

    # Creates security group for ELB
    @timeit
    def _create_sec_group_load_balancer(self):
        security_group = self.South_ec2_resource.create_security_group(
        Description='Allow inbound traffic',
        GroupName='load_balancer',
        # VpcId=self.South_vpc.id,
        TagSpecifications=[
                {
                    'ResourceType': 'security-group',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'load_balancer'
                        },
                        self.automation_tag,
                    ]
                },
            ],
        )

        security_group.authorize_ingress(
            CidrIp='0.0.0.0/0',
            FromPort=80,
            ToPort=80,
            IpProtocol='tcp',
        )

        security_group.load() # Commits updates
        self.load_balancer_sec_group = security_group
        return security_group

    # Creates instance for Django base AMI
    @timeit
    def _create_instance_django(self, sec_group):
        self.update_django_script()
        instances = self.South_ec2_resource.create_instances(
        BlockDeviceMappings=[{
                    'DeviceName': '/dev/xvda',
                    'Ebs': {
                        'DeleteOnTermination': True,
                        'VolumeSize': 8,
                        'VolumeType': 'gp2'
                    },
                },
        ],
        ImageId=str(self.ubuntu20amiSouth),
        InstanceType='t3.small',
        MaxCount=1,
        MinCount=1,
        KeyName="DIP_Ohio",
        # SubnetId=self.South_subnet.id,
        Monitoring={
            'Enabled': False
        },
        SecurityGroupIds=[
            sec_group.group_id,
        ],
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    self.automation_tag,
                    {
                        'Key': 'Name',
                        'Value': 'django',
                    },
                ],
            },
        ],
        UserData=self.script_django)
        instances[0].wait_until_running()
        return instances

    # Wrapper function to create MySQL server with proper configuration
    @timeit
    def create_db(self):

        # Sec group
        print("Creating mysql security group...")
        security_group=self._create_sec_group_db()

        # Make instance
        print("Creating mysql instance...")
        instances = self._create_instance_db(sec_group=security_group)
        instances[0].wait_until_running()
        print(f"Instance with mysql running on IP {self.get_db_ip()}:3306")
        return

    # Wrapper function to create Django instance to be used for creating AMI
    @timeit
    def create_django_base(self):

        # Sec group
        print("Creating django security group...")
        security_group=self._create_sec_group_django()

        # Make instance
        print("Creating django instance...")
        instances = self._create_instance_django(sec_group=security_group)
        instances[0].wait_until_running()
        time.sleep(90) # Wait for installation to finish, and reboot to take place.
        return

    # Extract AMI from running Django, then destroys it.
    @timeit
    def extract_django_image(self):
        instance = self.get_running_instances(self.South_ec2_resource)[0]
        self.django_AMI = instance.create_image(
            Name="django_orm_image",
            TagSpecifications=[{
                'ResourceType':'image',
                'Tags':[{'Key':'Name','Value':'django_orm_image'},self.automation_tag]
            }]
        )
        print("Creating django AMI...")
        self.django_AMI.wait_until_exists()
        state = self.django_AMI.state
        while state!='available':
            self.django_AMI.reload() # update attributes
            state = self.django_AMI.state
            time.sleep(1) # Wait a second
        # Done with image, kill instance
        instance.terminate()

    # Create autoscaling group from launch template that uses previously made AMI
    @timeit
    def create_auto_scaling_group(self):
        print("Creating launch template...")
        response = self.ec2Client.create_launch_template(
            LaunchTemplateName="django_template",
            LaunchTemplateData={
                'ImageId':self.django_AMI.image_id,
                'KeyName':"DIP_Ohio",
                'SecurityGroupIds':[self.django_sec_group.group_id],
                'InstanceType':'t3.micro',
                'Monitoring':{'Enabled': False },
                'BlockDeviceMappings':[{
                            'DeviceName': '/dev/xvda',
                            'Ebs': {
                                'DeleteOnTermination': True,
                                'VolumeSize': 8,
                                'VolumeType': 'gp2'
                            },
                        },
                        {
                            'DeviceName': '/dev/sda1',
                            'Ebs': {
                                'DeleteOnTermination': True,
                                'VolumeSize': 8,
                                'VolumeType': 'gp2'
                            },
                        },
                ],
            },
            TagSpecifications=[{
                'ResourceType':'launch-template',
                'Tags':[{'Key':'Name','Value':'django_template'},self.automation_tag]
            }]
        )
        self.launch_template_id = response['LaunchTemplate']['LaunchTemplateId']

        print("Creating autoscaling group...")
        self.asg = self.asgClient.create_auto_scaling_group(
            AutoScalingGroupName='asg_django',
            LaunchTemplate={
                'LaunchTemplateId': self.launch_template_id
            },
            MinSize=1,
            MaxSize=3,
            DesiredCapacity=2,
            DefaultCooldown=120,
            HealthCheckType='EC2',
            HealthCheckGracePeriod=60,
            AvailabilityZones = ['us-east-2a', 'us-east-2b', 'us-east-2c'],
            Tags=[{'Key':'Name', 'Value':'asg_django'}, self.automation_tag]
        )

    # Create ELB that targets autoscaling group
    @timeit
    def create_elastic_load_balancer(self):
        print("Creating load balancer security group...")
        sec_group = self._create_sec_group_load_balancer()
        
        print("Creating load balancer target group...")
        response = self.elbClient.create_target_group(
            Name='django-elb-tg',
            Protocol='HTTP',
            Port=8080,
            HealthCheckEnabled=True,
            HealthCheckProtocol='HTTP',
            HealthCheckPort='8080',
            HealthCheckPath='/tasks/',
            HealthCheckIntervalSeconds=120,
            HealthCheckTimeoutSeconds=30,
            TargetType='instance',
            VpcId = self.default_vpc_south,
            Tags=[
                {
                    'Key': 'Name',
                    'Value': 'django-elb-tg'
                },
                self.automation_tag
            ],
        )

        target_group_arn = response['TargetGroups'][0]['TargetGroupArn']

        print("Creating load balancer...")
        response = self.elbClient.create_load_balancer(
            Name= 'django-elb', 
            Subnets= self.get_available_subnets(), 
            Scheme= 'internet-facing', 
            Type= 'application',
            SecurityGroups=[sec_group.group_id],
            Tags=[
                {
                    'Key': 'Name',
                    'Value': 'django-elb'
                },
                self.automation_tag
            ]
        )

        load_balancer_arn = response['LoadBalancers'][0]['LoadBalancerArn']

        print("Attaching load balancer target group...")
        self.asgClient.attach_load_balancer_target_groups(
            AutoScalingGroupName = "asg_django",
            TargetGroupARNs=[
                target_group_arn,
            ]
        )

        print("Creating load balancer listener...")
        self.elbClient.create_listener(
            DefaultActions=[
                {
                    'TargetGroupArn': target_group_arn,
                    'Type': 'forward',
                },
            ],
            LoadBalancerArn=load_balancer_arn,
            Port=80,
            Protocol='HTTP',
        )
        

    @timeit
    def create_networking(self):
        self.South_vpc = self.South_ec2_resource.create_vpc(
            CidrBlock='10.100.0.0/20',
            TagSpecifications=[
                {
                    'ResourceType': 'vpc',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'vpc_dip'
                        },
                        self.automation_tag,
                    ]
                },
            ],
        )
        self.South_subnet = self.South_ec2_resource.create_subnet(
            CidrBlock='10.100.0.0/24',  # '10.100.0.0/24' has 256 addresses (254, excluding router and broadcast). Next 254 would be '10.100.1.0/24'
            TagSpecifications=[
                {
                    'ResourceType': 'subnet',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'subnet_dip'
                        },
                        self.automation_tag,
                    ]
                },
            ],
            VpcId=self.South_vpc.id
        )

    # Main procedure call. Asks for permission to delete all, then runs setup scripts in order.
    @timeit
    def construct_ORM(self):
        self.ask_delete_all()
        # If allowed, delete all, else return (can't make omelete without breaking eggs)
        if self.delete_all: 
            self.force_delete_all() 
        else: 
            return

        # self.create_networking()
        self.create_db()
        self.create_django_base()
        self.extract_django_image()
        self.create_auto_scaling_group()
        self.create_elastic_load_balancer()


cloud = CloudHandler()
cloud.construct_ORM()

In [31]:
cloud.force_delete_all()


Deleting db...
Destroying MySQL instance...
Destroying MySQL security group...
Finished delete_db in 49.68s
Finished delete_autoscaling_group in 1.54s
Destroying django base instance...
Destroying django_orm AMIs...
Destroying django security group...
Finished delete_django in 50.78s
Failed to delete target group, could be ok.
Finished delete_elastic_load_balancer in 1.92s
